In [35]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging
import matplotlib.pyplot as plt

import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 4 # 6 minutes hour of simulation
params.nP = 50 # reuqests (and passengers)
params.nV = 20 # vehicles



In [37]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.5
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



In [38]:
params.kpi = 3

In [39]:
inData = ExMAS.main(inData, params.shareability, plot=False) # create shareability graph (ExMAS) 

11-07-23 22:16:37-INFO-Initializing pairwise trip shareability between 50 and 50 trips.
11-07-23 22:16:37-INFO-creating combinations
11-07-23 22:16:37-INFO-2450	 nR*(nR-1)
11-07-23 22:16:37-INFO-Reduction of feasible pairs by 99.63%
11-07-23 22:16:37-INFO-Degree 2 	Completed
11-07-23 22:16:37-INFO-trips to extend at degree 2 : 37
11-07-23 22:16:37-INFO-At degree 2 feasible extensions found out of 0 searched
11-07-23 22:16:37-INFO-Degree 3 	Completed
11-07-23 22:16:37-INFO-No more trips to exted at degree 3


In [40]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

11-07-23 22:16:38-INFO-Matching 50 trips to 87 rides in order to min u_veh
11-07-23 22:16:38-INFO-Problem solution: Optimal. 
Total costs for single trips:         13,771 
reduced by matching to:               12,776


In [41]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],5.648500,305,1,[5.6485],"[0, 305]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44833169, 44767578]","[None, 0, 0]",node time req_id od 0 NaN...,305,3.05,4.581000,0.916200,3.6648
1,[1],1.584500,85,1,[1.5844999999999998],"[2, 85]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 3452839329, 44752059]","[None, 1, 1]",node time req_id od 0 ...,85,0.85,1.287000,0.257400,1.0296
2,[2],6.115500,330,1,[6.1155],"[91, 330]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1679761106, 44835816]","[None, 2, 2]",node time req_id od 0 ...,330,3.30,4.960500,0.992100,3.9684
3,[3],5.019500,271,1,[5.0195],"[277, 271]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 5591082257, 1322507105]","[None, 3, 3]",node time req_id od 0 ...,271,2.71,4.071000,0.814200,3.2568
4,[4],3.055500,165,1,[3.0555000000000003],"[457, 165]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1269112834, 1402771601]","[None, 4, 4]",node time req_id od 0 ...,165,1.65,2.478000,0.495600,1.9824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,"[15, 17]",8.628675,441,21,"[7.0462, 1.582475]","[889.0, 149, 89, 203]","[15, 17]","[17, 15]",2,82,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1608998306, 44787298, 44813101, 44859966]","[None, 15, 17, 17, 15]",node time req_id od 0 ...,441,4.41,5.871375,1.174275,4.6971
83,"[18, 19]",8.812300,451,21,"[6.343300000000001, 2.4690000000000003]","[1205.0, 140, 140, 171]","[18, 19]","[19, 18]",2,83,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1552650019, 4217813223, 44854271, 49865...","[None, 18, 19, 19, 18]",node time req_id od 0 ...,451,4.51,5.622750,1.124550,4.4982
84,"[21, 27]",10.261675,635,21,"[7.7635749999999994, 2.4981]","[1574.5, 386, 153, 96]","[21, 27]","[27, 21]",2,84,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44823653, 4349049085, 44767106, 44789407]","[None, 21, 27, 27, 21]",node time req_id od 0 ...,635,6.35,6.526125,1.305225,5.2209
85,"[36, 38]",8.476275,431,21,"[6.230625, 2.24565]","[2463.5, 274, 134, 23]","[36, 38]","[38, 36]",2,85,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44722228, 626009973, 44828189, 44819450]","[None, 36, 38, 38, 36]",node time req_id od 0 N...,431,4.31,5.664375,1.132875,4.5315


In [43]:
sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate


0.5
11-07-23 22:17:00-CRITICAL-this is request 0 with [0] available rides.
11-07-23 22:17:01-CRITICAL-ride 0 available [1]
11-07-23 22:17:01-CRITICAL-this is reuqest 0 with [0] still available rides.
hell
0.5
11-07-23 22:17:01-CRITICAL-this is request 0 with [0] available rides.
11-07-23 22:17:01-CRITICAL-ride 0 available [1]
11-07-23 22:17:01-CRITICAL-this is reuqest 0 with [0] still available rides.
hell
0.5
11-07-23 22:17:01-CRITICAL-this is request 0 with [0] available rides.
11-07-23 22:17:01-CRITICAL-ride 0 available [1]
11-07-23 22:17:01-CRITICAL-this is reuqest 0 with [0] still available rides.
hell
0.5
11-07-23 22:17:01-CRITICAL-this is request 0 with [0] available rides.
11-07-23 22:17:01-CRITICAL-ride 0 available [1]
11-07-23 22:17:01-CRITICAL-this is reuqest 0 with [0] still available rides.
hell
0.5
11-07-23 22:17:01-CRITICAL-this is request 0 with [0] available rides.
11-07-23 22:17:01-CRITICAL-ride 0 available [1]
11-07-23 22:17:01-CRITICAL-this is reuqest 0 with [0] sti

IndexError: single positional indexer is out-of-bounds

In [ ]:
sim.vehs[1].rdf 

In [ ]:
pd.DataFrame(sim.vehs[1].myrides)